In [5]:
(<$<) :: (a -> b) -> a -> b
(<$<) = ($)

(>$>) :: a -> (a -> b) -> b
x >$> f = f x
infixl 0 >$>

In [6]:
(\x -> 2 * x + 1) <$< 1
1 >$> (\x -> 2 * x + 1)

3

3

In [7]:
(<.<) :: (b -> c) -> (a -> b) -> (a -> c)
(<.<) = (.)

(>.>) :: (a -> b) -> (b -> c) -> (a -> c)
f >.> g = g . f
infixl 9 >.>

In [8]:
(+2) <.< (*3) <$< 5
(+2) >.> (*3) <$< 5

5 >$> (+2) <.< (*3)
5 >$> (+2) >.> (*3)

3 >$> (+2) >$> (+10)

17

21

17

21

15

In [10]:
tail [1,2]
tail >.> tail >.> tail $ [1,2]

[2]

: 

In [11]:
safeTail :: [a] -> Maybe [a]
safeTail []     = Nothing
safeTail (x:xs) = Just xs

In [14]:
safeTail [1,2]
safeTail []

--safeTail >.> safeTail >.> safeTail <$< [1,2]
--[1,2] >$> safeTail >.> safeTail >.> safeTail

:t safeTail

Just [2]

Nothing

safeTail :: forall a. [a] -> Maybe [a]

In [15]:
extractMaybe :: Maybe a -> a
extractMaybe Nothing  = error "Nothing inside!"
extractMaybe (Just x) = x

In [17]:
[1,2] >$> safeTail >.> extractMaybe >.> safeTail
[1,2] >$> safeTail >.> extractMaybe >.> safeTail >.> extractMaybe >.> safeTail

[1] >$> safeTail >.> extractMaybe >.> safeTail >.> extractMaybe >.> safeTail

Just []

Nothing

: 

In [18]:
insertMaybe :: a -> Maybe a
insertMaybe = Just

-- (>^$>) = extract (^) and apply ($)
(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b
ma >^$> f = (extractMaybe ma) >$> f
infixl 1 >^$>

Line 6: Redundant bracket
Found:
(extractMaybe ma) >$> f
Why not:
extractMaybe ma >$> f

In [21]:
insertMaybe [1,2]
[1,2] >$> insertMaybe >.> extractMaybe

--[1,2] >^$> safeTail
:t (>^$>)
insertMaybe [1,2] >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail
--insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail

Just [1,2]

[1,2]

(>^$>) :: forall a b. Maybe a -> (a -> Maybe b) -> Maybe b

Just [2]

Just []

Nothing

In [22]:
(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b
Nothing  >^$> _ = Nothing
(Just x) >^$> f = f x
infixl 1 >^$>

In [23]:
insertMaybe [1,2] >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail

Just [2]

Just []

Nothing

Nothing

In [24]:
f1 :: (Ord a, Num a) => a -> Maybe a
f1 x = if x > 0 then Just (x + 1) else Nothing

f2 :: (Eq a, Num a) => a -> Maybe a
f2 x = if x /= 0 then Just (10 * x) else Nothing

-- Kleisli composition
(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> g (extractMaybe (f x))

Line 9: Avoid lambda
Found:
\ x -> g (extractMaybe (f x))
Why not:
g . extractMaybe . f

In [26]:
h = f1 >.>> f2
:t h
f1 3
f2 3
h 3

let f3 x = Just (1/x)
let f4 x = Just (2 * x)
:t f3
:t f4

let f34 = f3 >.>> f4
:t f34
f34 2

h :: forall c. (Ord c, Num c) => c -> Maybe c

Just 4

Just 30

Just 40

f3 :: forall a. Fractional a => a -> Maybe a

f4 :: forall a. Num a => a -> Maybe a

f34 :: forall c. Fractional c => c -> Maybe c

Just 1.0

# Zadanie 1.1

In [27]:
f >.>> g = \x -> g (extractMaybe (f x))

Line 1: Avoid lambda
Found:
\ x -> g (extractMaybe (f x))
Why not:
g . extractMaybe . f

In [31]:
f >.>> g = \x -> g >^$> f x

# Zadanie 1.3

Możliwe jest napisanie funkcji ExtractIO, która pozwala na uzyskanie IO zapakowane w monadę.

In [33]:
:i Maybe
return 1 :: Maybe Int

Just 1

data Maybe a = Nothing | Just a 	-- Defined in ‘GHC.Maybe’
instance Functor Maybe -- Defined in ‘GHC.Base’
instance Show a => Show (Maybe a) -- Defined in ‘GHC.Show’
instance Semigroup a => Monoid (Maybe a) -- Defined in ‘GHC.Base’
instance Semigroup a => Semigroup (Maybe a) -- Defined in ‘GHC.Base’
instance Eq a => Eq (Maybe a) -- Defined in ‘GHC.Maybe’
instance Ord a => Ord (Maybe a) -- Defined in ‘GHC.Maybe’
instance Read a => Read (Maybe a) -- Defined in ‘GHC.Read’
instance Applicative Maybe -- Defined in ‘GHC.Base’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Traversable Maybe -- Defined in ‘Data.Traversable’

In [34]:
:i Monad

class Applicative m => Monad (m :: * -> *) where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
  fail :: String -> m a
  {-# MINIMAL (>>=) #-}
  	-- Defined in ‘GHC.Base’
instance Monad (Either e) -- Defined in ‘Data.Either’
instance Monad IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Monad [] -- Defined in ‘GHC.Base’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance Monad IO -- Defined in ‘GHC.Base’
instance Monad ((->) r) -- Defined in ‘GHC.Base’
instance Monoid a => Monad ((,) a) -- Defined in ‘GHC.Base’
instance (Monoid a, Monoid b, Monoid c) => Monad ((,,,) a b c) -- Defined in ‘Data.Orphans’
instance (Monoid a, Monoid b) => Monad ((,,) a b) -- Defined in ‘Data.Orphans’

In [36]:
import Control.Monad
:t (>=>)

(>=>) :: forall (m :: * -> *) a b c. Monad m => (a -> m b) -> (b -> m c) -> a -> m c

In [37]:
insertMaybe [1,2]
return (Just [1,2])
return [1,2]
return [1,2] :: Maybe [Int]

insertMaybe [1,2] >^$> safeTail
return [1,2] >>= safeTail
(return [1,2] >>= safeTail) == (insertMaybe [1,2] >^$> safeTail)

insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail
return [1,2] >>= safeTail >>= safeTail >>= safeTail >>= safeTail

return [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail
insertMaybe [1,2] >>= safeTail >^$> safeTail >>= safeTail >^$> safeTail
return [1,2] >^$> safeTail >>= safeTail >^$> safeTail >>= safeTail



Just [1,2]

Just [1,2]

[1,2]

Just [1,2]

Just [2]

...

Nothing

Nothing

Nothing

Nothing

Nothing

In [38]:
doSafeTail3x :: [a] -> Maybe [a]
doSafeTail3x xs = do
  t1 <- safeTail xs
  t2 <- safeTail t1
  t3 <- safeTail t2
  return t3

safeTail3x :: [a] -> Maybe [a]
safeTail3x xs =
  safeTail xs >>= \t1 ->
    safeTail t1 >>= \t2 ->
      safeTail t2 >>= \t3 ->
        return t3

safeTail3x' :: [a] -> Maybe [a]
safeTail3x' xs = return xs >>= safeTail >>= safeTail >>= safeTail

Line 2: Redundant return
Found:
do t1 <- safeTail xs
   t2 <- safeTail t1
   t3 <- safeTail t2
   return t3
Why not:
do t1 <- safeTail xs
   t2 <- safeTail t1
   safeTail t2Line 10: Use >=>
Found:
\ t1 -> safeTail t1 >>= \ t2 -> safeTail t2 >>= \ t3 -> return t3
Why not:
(safeTail Control.Monad.>=>
   (\ t2 -> safeTail t2 >>= \ t3 -> return t3))Line 11: Use >=>
Found:
\ t2 -> safeTail t2 >>= \ t3 -> return t3
Why not:
(safeTail Control.Monad.>=> (\ t3 -> return t3))Line 16: Monad law, left identity
Found:
return xs >>= safeTail
Why not:
safeTail xs

In [39]:
doSafeTail3x [1..5]
safeTail3x [1..5]
safeTail3x' [1..5]

doSafeTail3x [1]
safeTail3x [1]
safeTail3x' [1]

Just [4,5]

Just [4,5]

Just [4,5]

Nothing

Nothing

Nothing

In [40]:
f5 :: Int -> Int -> Int -> Int
f5 x y z = 1000 `div` x + 100 `div` y + 10 `div` z

In [43]:
f5 1 2 3
f5 1 2 0
f5 1 0 3
f5 0 2 3

1053

: 

In [44]:
safeDiv :: Int -> Int -> Maybe Int
safeDiv x y | y /= 0    = Just $ x `div` y
            | otherwise = Nothing

safeF5 :: Int -> Int -> Int -> Maybe Int
safeF5 x y z =
  case (safeDiv 1000 x) of
    Nothing -> Nothing
    Just (iOverX) ->
      case (safeDiv 100 y) of
        Nothing -> Nothing
        Just (iOverY) ->
          case (safeDiv 10 z) of
            Nothing -> Nothing
            Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ

Line 7: Redundant bracket
Found:
case (safeDiv 1000 x) of
    Nothing -> Nothing
    Just (iOverX) -> case (safeDiv 100 y) of
                         Nothing -> Nothing
                         Just (iOverY) -> case (safeDiv 10 z) of
                                              Nothing -> Nothing
                                              Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 1000 x of
    Nothing -> Nothing
    Just (iOverX) -> case (safeDiv 100 y) of
                         Nothing -> Nothing
                         Just (iOverY) -> case (safeDiv 10 z) of
                                              Nothing -> Nothing
                                              Just (iOverZ) -> Just $ iOverX + iOverY + iOverZLine 9: Redundant bracket
Found:
(iOverX)
Why not:
iOverXLine 10: Redundant bracket
Found:
case (safeDiv 100 y) of
    Nothing -> Nothing
    Just (iOverY) -> case (safeDiv 10 z) of
                         Nothing -> Nothing
                         Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 100 y of
    Nothing -> Nothing
    Just (iOverY) -> case (safeDiv 10 z) of
                         Nothing -> Nothing
                         Just (iOverZ) -> Just $ iOverX + iOverY + iOverZLine 12: Redundant bracket
Found:
(iOverY)
Why not:
iOverYLine 13: Redundant bracket
Found:
case (safeDiv 10 z) of
    Nothing -> Nothing
    Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 10 z of
    Nothing -> Nothing
    Just (iOverZ) -> Just $ iOverX + iOverY + iOverZLine 15: Redundant bracket
Found:
(iOverZ)
Why not:
iOverZ

In [45]:
safeF5 1 2 3
safeF5 1 2 0
safeF5 1 0 3
safeF5 0 2 3

Just 1053

Nothing

Nothing

Nothing

In [46]:
safeF5' :: Int -> Int -> Int -> Maybe Int
safeF5' x y z = do
  iOverX <- safeDiv 1000 x
  iOverY <- safeDiv 100 y
  iOverZ <- safeDiv 10 z
  return $ iOverX + iOverY + iOverZ

In [47]:
safeF5' 1 2 3
safeF5' 1 2 0
safeF5' 1 0 3
safeF5' 0 2 3

Just 1053

Nothing

Nothing

Nothing

# Zadanie 2.1

In [17]:
import Data.Maybe
(>=>) :: Maybe a -> (a -> Maybe b) -> Maybe b
Nothing >=> _ = Nothing
(Just x) >=> f = f x

# Zadanie 2.3

In [2]:
join :: Maybe (Maybe a) -> Maybe a
join (Just (Just a)) = Just a
join (Just Nothing)  = Nothing
join Nothing         = Nothing

# Zadanie 2.2

In [6]:
import Data.Maybe
:i Maybe

data Maybe a = Nothing | Just a 	-- Defined in ‘GHC.Maybe’
instance Functor Maybe -- Defined in ‘GHC.Base’
instance Show a => Show (Maybe a) -- Defined in ‘GHC.Show’
instance Semigroup a => Monoid (Maybe a) -- Defined in ‘GHC.Base’
instance Semigroup a => Semigroup (Maybe a) -- Defined in ‘GHC.Base’
instance Eq a => Eq (Maybe a) -- Defined in ‘GHC.Maybe’
instance Ord a => Ord (Maybe a) -- Defined in ‘GHC.Maybe’
instance Read a => Read (Maybe a) -- Defined in ‘GHC.Read’
instance Applicative Maybe -- Defined in ‘GHC.Base’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Traversable Maybe -- Defined in ‘Data.Traversable’

In [18]:
:i []

[1,2,3] :: [Int]
[1,2,3] :: [] Int

[1,2,3]

[1,2,3]

data [] a = [] | a : [a] 	-- Defined in ‘GHC.Types’
instance IHaskellDisplay a => IHaskellDisplay [a] -- Defined in ‘IHaskell.Types’
instance Functor [] -- Defined in ‘GHC.Base’
instance Show a => Show [a] -- Defined in ‘GHC.Show’
instance Monoid [a] -- Defined in ‘GHC.Base’
instance Semigroup [a] -- Defined in ‘GHC.Base’
instance Eq a => Eq [a] -- Defined in ‘GHC.Classes’
instance Ord a => Ord [a] -- Defined in ‘GHC.Classes’
instance Read a => Read [a] -- Defined in ‘GHC.Read’
instance Applicative [] -- Defined in ‘GHC.Base’
instance Monad [] -- Defined in ‘GHC.Base’
instance Foldable [] -- Defined in ‘Data.Foldable’
instance Traversable [] -- Defined in ‘Data.Traversable’

In [19]:
return 1 :: [] Int
return 3 >>= (\x -> [1..x])
[1,2] >>= (\x -> [-x,x])
[1,2,3] >>= (\x -> [-x..x])

[1,2] >>= (\x -> [-x,x]) >>= (\y -> [-y,y])
[1,2] >>= \x -> [-x,x] >>= \y -> [-y,y]
[1,2] >>= \x -> [-x,x] >>= \y -> return (x,y)

[1]

[1,2,3]

[-1,1,-2,2]

[-1,0,1,-2,-1,0,1,2,-3,-2,-1,0,1,2,3]

[1,-1,-1,1,2,-2,-2,2]

[1,-1,-1,1,2,-2,-2,2]

[(1,-1),(1,1),(2,-2),(2,2)]

In [20]:
xs1 :: [(Int,Int,Int)]
xs1 = [ (x,y,z) | let xs = [1,2], x <- xs, y <- xs, z <- xs ]

doXs1 :: [(Int,Int,Int)]
doXs1 = do
  let xs = [1,2]
  x <- xs
  y <- xs
  z <- xs
  return (x,y,z)

In [21]:
xs1
doXs1
xs1 == doXs1

[(1,1,1),(1,1,2),(1,2,1),(1,2,2),(2,1,1),(2,1,2),(2,2,1),(2,2,2)]

[(1,1,1),(1,1,2),(1,2,1),(1,2,2),(2,1,1),(2,1,2),(2,2,1),(2,2,2)]

True

In [23]:
import Control.Monad
xs2 :: [(Int,Int,Int)]
xs2 = [ (x,y,z) | let xs = [1..3], x <- xs, y <- xs, z <- xs, x > y && y > z ]

doXs2 :: [(Int,Int,Int)]
doXs2 = do
  let xs = [1..3]
  x <- xs
  y <- xs
  z <- xs
  guard $ x > y && y > z
  return (x,y,z)

doXs2' :: [(Int,Int,Int)]
doXs2' = do
  let xs = [1..3]
  x <- xs
  y <- xs
  z <- xs
  if x > y && y > z
    then return (x,y,z)
    else []

Line 7: Reduce duplication
Found:
let xs = [1 .. 3]
x <- xs
y <- xs
z <- xs

Why not:
Combine with -:16:3

In [24]:
:t guard
[1..10] >>= \x -> guard (even x) >> return x
[1..10] >>= \x -> guard (even x) >>= \y -> return (x,y)

[1..10] >>= \x -> return x
[1..10] >>= return

guard :: forall (f :: * -> *). Alternative f => Bool -> f ()

[2,4,6,8,10]

[(2,()),(4,()),(6,()),(8,()),(10,())]

[1,2,3,4,5,6,7,8,9,10]

[1,2,3,4,5,6,7,8,9,10]

# Zadanie 4.2

In [32]:
join :: [[a]] -> [a]
join xss = foldr (++) [] xss

Line 2: Eta reduce
Found:
join xss = foldr (++) [] xss
Why not:
join = foldr (++) []Line 2: Use concat
Found:
foldr (++) []
Why not:
concat

# Zadanie 4.1

Dostępne metody w ramach implementacji Monad []

fmap :: (a -> b) -> [a] -> [b]

(<$) :: a -> [b] -> [a]